# STUNIR Advanced Features Tutorial

This notebook covers advanced STUNIR capabilities:

1. **Multi-Target Generation**: Emit code for multiple languages
2. **Custom Emitters**: Create your own code generators
3. **Manifest System**: Track and verify all artifacts
4. **Pipeline Orchestration**: Automate the build process

In [ ]:
import json
import hashlib
from datetime import datetime, timezone
from typing import Dict, Any, List
from dataclasses import dataclass
from enum import Enum
from abc import ABC, abstractmethod

def canonical_json(data: Any) -> str:
    return json.dumps(data, sort_keys=True, separators=(',', ':'), ensure_ascii=False)

def compute_sha256(data: str) -> str:
    return hashlib.sha256(data.encode('utf-8')).hexdigest()

print('Setup complete!')

## 1. Multi-Target Code Generation

In [ ]:
class TargetLanguage(Enum):
    PYTHON = 'python'
    RUST = 'rust'
    C99 = 'c99'
    GO = 'go'

TYPE_MAPPINGS = {
    TargetLanguage.PYTHON: {'i32': 'int', 'f64': 'float', 'bool': 'bool'},
    TargetLanguage.RUST: {'i32': 'i32', 'f64': 'f64', 'bool': 'bool'},
    TargetLanguage.C99: {'i32': 'int32_t', 'f64': 'double', 'bool': 'bool'},
    TargetLanguage.GO: {'i32': 'int32', 'f64': 'float64', 'bool': 'bool'},
}

ir = {
    'module': {'name': 'math_utils', 'version': '1.0.0'},
    'functions': [
        {'name': 'square', 'params': [{'name': 'x', 'type': 'i32'}], 'returns': 'i32'},
        {'name': 'sqrt', 'params': [{'name': 'x', 'type': 'f64'}], 'returns': 'f64'},
    ]
}

def emit_python(func: Dict) -> str:
    params = ', '.join(f"{p['name']}: {TYPE_MAPPINGS[TargetLanguage.PYTHON].get(p['type'], 'Any')}" 
                       for p in func.get('params', []))
    ret = TYPE_MAPPINGS[TargetLanguage.PYTHON].get(func.get('returns', 'void'), 'Any')
    return f"def {func['name']}({params}) -> {ret}:\n    pass\n"

print('=== Python Output ===')
for func in ir['functions']:
    print(emit_python(func))

## 2. Custom Emitter Framework

In [ ]:
class BaseEmitter(ABC):
    @abstractmethod
    def language_name(self) -> str: pass
    
    @abstractmethod
    def emit_function(self, func: Dict) -> str: pass
    
    def emit(self, ir: Dict) -> str:
        parts = [f"// Generated {self.language_name()} module"]
        for func in ir.get('functions', []):
            parts.append(self.emit_function(func))
        return '\n'.join(parts)

class TypeScriptEmitter(BaseEmitter):
    TYPE_MAP = {'i32': 'number', 'f64': 'number', 'bool': 'boolean'}
    
    def language_name(self) -> str: return 'TypeScript'
    
    def emit_function(self, func: Dict) -> str:
        params = ', '.join(f"{p['name']}: {self.TYPE_MAP.get(p['type'], 'any')}" 
                           for p in func.get('params', []))
        ret = self.TYPE_MAP.get(func.get('returns', 'void'), 'any')
        return f"export function {func['name']}({params}): {ret} {{ throw new Error('Not implemented'); }}"

ts_emitter = TypeScriptEmitter()
print('=== TypeScript Output ===')
print(ts_emitter.emit(ir))

## 3. Manifest System

In [ ]:
@dataclass
class ManifestEntry:
    name: str
    path: str
    hash: str
    size: int

class ManifestGenerator:
    def __init__(self):
        self.entries: List[ManifestEntry] = []
    
    def add_artifact(self, name: str, content: str):
        self.entries.append(ManifestEntry(
            name=name,
            path=f'output/{name}',
            hash=compute_sha256(content),
            size=len(content.encode('utf-8'))
        ))
    
    def generate(self) -> Dict[str, Any]:
        manifest = {
            'schema': 'stunir.manifest.targets.v1',
            'entries': [{'name': e.name, 'hash': e.hash[:16]} for e in self.entries]
        }
        manifest['manifest_hash'] = compute_sha256(canonical_json(manifest))[:16]
        return manifest

gen = ManifestGenerator()
gen.add_artifact('demo.ts', ts_emitter.emit(ir))
print('=== Manifest ===')
print(json.dumps(gen.generate(), indent=2))

## Summary

You learned:
- Multi-target code generation
- Creating custom emitters
- Using the manifest system

Next: Explore the Python examples in `examples/python/`